In [16]:
import json
from pyvis.network import Network
import networkx as nx

# Load the JSON file
with open('data/updated_data_plus.json', 'r', encoding='utf-8') as file:
    data = json.load(file)['chunks']

# Create a mapping of id to entry for edges and fast lookup
id_to_entry = {entry["id"]: entry for entry in data}

# Build a NetworkX directed graph
G = nx.DiGraph()

# Add nodes and edges to the graph
for entry in data:
    node_id = entry["id"]  # Use `id` as a unique identifier for each node
    G.add_node(node_id, label=entry["title"])
    for target_id in entry.get("relevantChunksIds", []):
        if target_id in id_to_entry:  # Ensure target node exists
            G.add_edge(node_id, target_id)

# Function to determine node color based on title prefix
def determine_color(title):
    if title.startswith("ErwG"):
        return "violet"
    elif title.startswith("Art"):
        return "SpringGreen"
    elif title.startswith("Anhang"):
        return "SkyBlue"
    else:
        return "gray"

# Initialize a Pyvis network
net = Network(notebook=True, directed=True, height="900px", width="100%", cdn_resources="remote", select_menu=True, filter_menu=True)

# Add nodes with additional data
for entry in data:
    node_id = entry["id"]  # Unique node identifier
    title = entry["title"]
    color = determine_color(title)
    
    # Get parameters for size and hover text
    content_length = entry["content_length"]
    content_plus_length = entry["content_plus_length"]
    relevantChunksIdsNum = entry["relevantChunksIdsNum"]
    
    hover_text = (
        f"Title: {title} | Content Length: {content_length} | "
        f"Content Plus Length: {content_plus_length} | "
        f"Relevant Chunks IDs Num: {relevantChunksIdsNum}"
    )
    
    # Initial size is based on content_length, but can be controlled dynamically
    size = 10 + content_length / 100

    # Add the node with unique ID and hover text
    net.add_node(
        node_id,
        label=title,  # Display title and ID for clarity
        title=hover_text,
        color={"background": color, "border": "black"},
        size=size,
        content_length=content_length,
        content_plus_length=content_plus_length,
        relevantChunksIdsNum=relevantChunksIdsNum,
    )

# Add edges
for edge in G.edges:
    source, target = edge
    net.add_edge(source, target, title="relates to", color="black", borderWidth=2)

# Add PyVis options
net.set_options("""
{
  "physics": {
    "repulsion": {
      "centralGravity": 0,
      "springLength": 380,
      "nodeDistance": 210
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  },
  "interaction": {
    "tooltipDelay": 200
  },
  "edges": {
    "color": {
      "inherit": true
    }
  }
}
""")

# Inject JavaScript for dropdown functionality
dropdown_script = """
<script>
(function() {
    function resizeNodes(sizeBy) {
        const sizeFactor = {
            "content_length": 100,
            "content_plus_length": 500,
            "relevantChunksIdsNum": 0.5
        };

        network.body.data.nodes.update(
            network.body.data.nodes.get().map(node => {
                node.size = 10 + node[sizeBy] / sizeFactor[sizeBy];
                return node;
            })
        );
    }

    // Create dropdown menu
    const dropdown = document.createElement("select");
    dropdown.style.margin = "10px";
    dropdown.innerHTML = `
        <option value="content_length">Content Length</option>
        <option value="content_plus_length">Content + Length</option>
        <option value="relevantChunksIdsNum">Relevant Chunks Number</option>
    `;
    dropdown.onchange = function() {
        resizeNodes(this.value);
    };
    document.body.prepend(dropdown);
})();
</script>
"""

# Show the graph and append the dropdown script
output_file = "interactive_graph.html"
net.show(output_file)

# Append the dropdown script to the generated HTML file
with open(output_file, "a", encoding="utf-8") as file:
    file.write(dropdown_script)


interactive_graph.html
